### Step 1: Import Data

In [41]:
from sklearn.datasets import fetch_openml
import numpy as np

# Import MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
x_full = mnist.data.values  # Full dataset
y_full = mnist.target.values.astype(int)  # Labels (0–9)

print(f"x_full shape: {x_full.shape}, y_full shape: {y_full.shape}")

/opt/conda/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


x_full shape: (70000, 784), y_full shape: (70000,)


### Step 2: Create Subset

In [42]:
# Create a subset of training data (1000 images per digit)
subset_size = 100
x_subset = []
y_subset = []

for digit in range(10):
    digit_indices = np.where(y_full == digit)[0][:subset_size]
    x_subset.append(x_full[digit_indices])
    y_subset.append(y_full[digit_indices])

x_subset = np.vstack(x_subset)
y_subset = np.hstack(y_subset)

print(f"x_subset shape: {x_subset.shape}, y_subset shape: {y_subset.shape}")
print(f"Unique labels in y_subset: {np.unique(y_subset)}")
print(f"x_subset[3].shape: {x_subset[3].shape}")

x_subset shape: (1000, 784), y_subset shape: (1000,)
Unique labels in y_subset: [0 1 2 3 4 5 6 7 8 9]
x_subset[3].shape: (784,)


### Step 3: Apply PCA

In [53]:
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensionality
n_components = 77
  # Adjustable based on explained variance
pca = PCA(n_components=n_components)
x_pca = pca.fit_transform(x_subset)

# # Load PCA instance
# with open("models/pca_model.pkl", "wb") as file:
#     pickle.dump(pca, file)  # Save the fitted PCA model

# Load PCA instance
with open(f"{models_dir}pca_instance.pkl", "rb") as file:
    pca = pickle.load(file)

print(f"Original shape: {x_subset.shape}, PCA shape: {x_pca.shape}")
print(f"variance retained: {np.sum(pca.explained_variance_ratio_)*100}%")

print(f"shape of x_pca: ", x_pca.shape)
print(f"shape of x_pca[0]: ", x_pca[0].shape)

print(f"check any image vector: ", x_pca[999])

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/models/pca_instance.pkl'

### Step 4: Preprocess Data

In [44]:
# Thresholding: Convert to binary
threshold_value = 0
x_b = (x_pca > threshold_value).astype(int)

print(f"x_b shape: {x_pca.shape}")
print(f"x_b[4] =", x_b[250])

x_b shape: (1000, 77)
x_b[4] = [1 1 0 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 0 0 0
 1 0 0 1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0 0
 1 1 1]


*Install CuPy for CUDA 12.2*

In [45]:
!pip install cupy-cuda12x

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


*Set folder*

In [46]:
import os

models_dir = "/kaggle/working/models/"  # Kaggle's default working directory

# Ensure the directory exists
os.makedirs(models_dir, exist_ok=True)

### Step 5: Train Classifiers

In [47]:
import pickle
import cupy as cp
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt

# Feasibility check function
def check_feasibility_and_compute_coefficients(z, x_b, y_binary):
    num_data_points = x_b.shape[0]
    num_coefficients = n_components + 1  # (+1 for constant term)
    delta = 1e-6  # a small positive value

    # Construct G(x) and H(x) matrices for numerator and denominator
    G = cp.zeros((num_data_points, num_coefficients))  # Numerator matrix
    H = cp.zeros((num_data_points, num_coefficients))  # Denominator matrix

    for i in range(num_data_points):
      G[i, 0] = 1
      H[i, 0] = 1
      for j in range(num_coefficients-1):
        G[i, j+1] = x_b[i, j] ** (j+1)
        H[i, j+1] = x_b[i, j] ** (j+1)

    # print(f"G: {G}")
    # print(f"H: {H}")

    # Construct constraints for Ax <= b
    A = []
    b = []

    for i in range(num_data_points):
        f_plus_z = y_binary[i] + z  # Upper bound
        f_minus_z = y_binary[i] - z  # Lower bound

        # Constraint 1: (f(xi) - z) * β^T H(xi) - α^T G(xi) ≤ θ
        # (-G(xi))αT + (f(xi) - z).H(xi)βT + (-1)θ ≤ 0
        constraint_1 = cp.zeros(2 * num_coefficients + 1)
        # (1) Coefficients of α
        constraint_1[0:num_coefficients] = -G[i]
        # (2) Coefficients of β
        constraint_1[num_coefficients:2 * num_coefficients] = (f_minus_z) * H[i]
        # (3) Coefficient of θ (last element)
        constraint_1[-1] = -1
        A.append(constraint_1)
        b.append(0)

        # Constraint 2: α^T G(xi) + (-1).(f(xi) + z) * β^T H(xi) ≤ θ
        # G(xi).αT + (-1)(f(xi) - z).H(xi)βT + (-1)θ ≤ 0
        constraint_2 = cp.zeros(2 * num_coefficients + 1)
        # (1) Coefficients of α
        constraint_2[0:num_coefficients] = G[i]
        # (2) Coefficients of β
        constraint_2[num_coefficients:2 * num_coefficients] = -(f_plus_z) * H[i]
        # (3) Coefficient of θ (last element)
        constraint_2[-1] = -1
        A.append(constraint_2)
        b.append(0)

        # Constraint 3: β^T H(x) ≥ δ
        # (0)α^T + (-H(x)) β^T + (0)θ ≤ -δ
        constraint_3 = cp.zeros(2 * num_coefficients + 1)
        # Coefficient of β
        constraint_3[num_coefficients:2 * num_coefficients] = -H[i]
        A.append(constraint_3)
        b.append(-delta)

    # Convert CuPy arrays to NumPy arrays for SciPy
    A = cp.asnumpy(cp.array(A))
    b = cp.asnumpy(cp.array(b))

    # print(f"A =", len(A))
    # print(f"A: {A[0]}")
    # print(f"len(A[0]): {len(A[0])}")
    # print(f"len(b): {len(b)}")

    # Objective function to minimize θ
    c = cp.asnumpy(cp.zeros(2 * num_coefficients + 1))
    c[-1] = 1  # Only θ has a coefficient in the objective function

    # Solve the linear programming problem (methods: highs, revised simplex)
    result = linprog(c, A_ub=A, b_ub=b, method="highs")

    # Check feasibility and return results
    if result.success:
        alpha_coefficients = result.x[:num_coefficients]
        beta_coefficients = result.x[num_coefficients:2 * num_coefficients]
        theta = result.x[-1]
        return True, alpha_coefficients, beta_coefficients, theta
    else:
        return False, None, None, None


# Bisection loop
def bisection_loop(x_b, y_binary, uL, uH, precision):
    optimal_alpha, optimal_beta, optimal_theta = None, None, None
    z_values = []

    while uH - uL > precision:
        z = (uL + uH) / 2
        z_values.append(z)
        feasible, alpha_coefficients, beta_coefficients, theta = check_feasibility_and_compute_coefficients(z, x_b, y_binary)

        if feasible:
            uH = z
            optimal_alpha, optimal_beta, optimal_theta = alpha_coefficients, beta_coefficients, theta
        else:
            uL = z

    return uH, optimal_alpha, optimal_beta, optimal_theta, z_values

# Train a classifier for each digit
for digit in range(10):
    print(f"Training classifier for digit {digit}...")

    # Assign labels: Positive for the current digit, negative for others
    y_binary = (y_subset == digit).astype(int)

    # Bisection parameters
    uL = 0  # Initial lower bound
    uH = 500  # Initial upper bound
    precision = 1e-2 #1e-6  # Precision threshold

    # Run bisection loop
    optimal_z, optimal_alpha, optimal_beta, optimal_theta, z_values = bisection_loop(x_b, y_binary, uL, uH, precision)

    # Print results
    print(f"Number of Iterations: {len(z_values)}")
    print(f"z Values in all Iterations: {z_values}")
    print(f"Optimal z (Maximum Deviation): {optimal_z}")

    # # Plot convergence of z values
    # plt.figure(figsize=(8, 6))
    # plt.plot(range(len(z_values)), z_values, marker='o', linestyle='-')
    # plt.xlabel("Iteration")
    # plt.ylabel("z Value")
    # plt.title("Convergence of z Values")
    # plt.grid(True)
    # plt.show()

    print(f"Optimized Coefficients (Numerator α): {optimal_alpha}")
    print(f"Optimized Coefficients (Denominator β): {optimal_beta}")
    print(f"Optimal θ: {optimal_theta}")
    # print(f"rational_function =", rational_function(x_b[0], optimal_alpha, optimal_beta))

    # Save the model
    model = {
        "alpha": optimal_alpha,
        "beta": optimal_beta,
        "theta": optimal_theta,
        "n_components": n_components
    }

    with open(f"{models_dir}classifier_{digit}.pkl", "wb") as file:
        pickle.dump(model, file)

    print(f"Model for digit {digit} saved at {models_dir}")

Training classifier for digit 0...
Number of Iterations: 16
z Values in all Iterations: [250.0, 125.0, 62.5, 31.25, 15.625, 7.8125, 3.90625, 1.953125, 0.9765625, 0.48828125, 0.244140625, 0.1220703125, 0.06103515625, 0.030517578125, 0.0152587890625, 0.00762939453125]
Optimal z (Maximum Deviation): 0.00762939453125
Optimized Coefficients (Numerator α): [ 5.e-07 -0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00  0.e+00  0.e+00  0.e+00
  0.e+00  0.e+00 -0.e+00 -0.e+00 -0.e+00  0.e+00 -0.e+00 -0.e+00 -0.e+00
 -0.e+00 -0.e+00 -0.e+00  0.e+00 -0.e+00 -0.e+00  0.e+00 -0.e+00 -0.e+00
 -0.e+00  0.e+00  0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00
 -0.e+00 -0.e+00 -0.e+00  0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00
 -0.e+00  0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00  0.e+00  0.e+00  0.e+00
 -0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00 -0.e+00  0.e+00  0.e+00 -0.e+00
  0.e+00 -0.e+00 -0.e+00  0.e+00 -0.e+00  0.e+00 -0.e+00 -0.e+00 -0.e+00
  0.e+00 -0.e+00  0.e+00 -0.e+00 -0.e+00 -0.e+00]
Optimized Coe

### Testing

In [49]:
import cupy as cp
import numpy as np
from sklearn.decomposition import PCA
import pickle
import matplotlib.pyplot as plt

# Define the rational function
def rational_function(x, alpha, beta):
    """
    r(x) = (α_0 + α_1*x1**1 + α_2*x2**2 + ...) / 
           (β_0 + β_1*x1**1 + β_2*x2**2 + ...).
    """
    numerator = alpha[0] + sum(alpha[i+1] * x[i] ** (i + 1) for i in range(len(x)))
    denominator = beta[0] + sum(beta[i+1] * x[i] ** (i + 1) for i in range(len(x)))
    return numerator / denominator

# Set the desired number of test images
desired_num_test_images = 1000  # Change this to any desired number

# Load MNIST test data (10,000 images)
from keras.datasets import mnist
(_, _), (x_test, y_test) = mnist.load_data()

print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")
# # Load the test dataset (use your MNIST data here)
# # Assuming x_test contains 10,000 test images
# x_test = np.random.rand(10000, 784)  # Replace this with actual MNIST test images
# y_test = np.random.randint(0, 10, 10000)  # Replace with actual test labels

# Subset the test dataset
x_test_subset = x_test[:desired_num_test_images]
y_test_subset = y_test[:desired_num_test_images]

print(f"Shape of original test subset: {x_test_subset.shape}")

# Load PCA instance
with open("models/pca_model.pkl", "rb") as file:
    pca = pickle.load(file)  # Load the PCA model trained on training data
x_test_pca = pca.transform(x_test_subset)  # Transform test data using the saved PCA

# # Apply PCA to test data
# n_components = 78  # Desired number of components
# pca = PCA(n_components=n_components)
# x_test_pca = pca.fit_transform(x_test_subset)

# Thresholding: Convert PCA-transformed data to binary (0s and 1s)
threshold_value = 0
x_test_binary = (x_test_pca > threshold_value).astype(int)

print("PCA applied to test data.")
print(f"Shape of PCA-transformed test subset: {x_test_pca.shape}")
print(f"Binary thresholded test subset: {x_test_binary.shape}")

# Load the saved models and test
models_dir = "/kaggle/working/models/"  # Update based on your environment
accuracies = []

for digit in range(10):
    # Load model for each digit
    with open(f"{models_dir}classifier_{digit}.pkl", "rb") as file:
        model = pickle.load(file)

    alpha = model["alpha"]
    beta = model["beta"]
    theta = model["theta"]

    # Evaluate the rational function for each test data point
    y_predicted = [
        rational_function(x, alpha, beta) for x in x_test_binary
    ]

    # Convert predictions to binary (1 for this digit, 0 for others)
    y_pred_binary = np.array(y_predicted) > 0.5
    y_true_binary = y_test_subset == digit

    # Calculate accuracy for this digit
    accuracy = np.mean(y_pred_binary == y_true_binary)
    accuracies.append(accuracy)

    print(f"Accuracy for digit {digit}: {accuracy*100:.2f}%")
    
# Calculate and print overall accuracy
overall_accuracy = np.mean(accuracies)
print(f"Overall Accuracy: {overall_accuracy:.2f}")

# Plotting accuracies for each digit
plt.bar(range(10), accuracies, color='blue', alpha=0.7, label="Accuracy")
plt.xlabel("Digits")
plt.ylabel("Accuracy (%)")
plt.title("Accuracy for Each Digit")
plt.xticks(range(10))
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.show()

x_test shape: (10000, 28, 28), y_test shape: (10000,)
Shape of original test subset: (1000, 28, 28)


ValueError: Found array with dim 3. PCA expected <= 2.